## Baixando Bibliotecas

In [ ]:
% pip install --upgrade pip
!pip install --disable-pip-version-check \
!pip install torch==1.13.1 torchdata==0.5.1 --quiet
!pip install transformers==4.27.2 datasets==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: Invalid requirement: '!pip': Expected package name at the start of dependency specifier
    !pip
    ^
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#importando as bibliotecas
from datasets import get_dataset_config_names, load_dataset, Dataset, DatasetDict, concatenate_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig,TrainingArguments, Trainer,  Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
import time
import pandas as pd
import numpy as np


## Treinando novo tokenizador
https://huggingface.co/learn/nlp-course/pt/chapter6/2?fw=tf

In [ ]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    )


model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
example = ('Última quarta-feira (4), guia turístico morto no Centro do Rio ao reagir a assalto praticado por duas mulheres. Imagens das câmeras de segurança do local mostram Daniel Mascarenhas Xavier da Silva, de 31 anos, ser abordado pelas duas mulheres que estavam em uma motocicleta. Elas pegaram sua bolsa e os pertences que a vítima tinha no bolso. Leia mais Neste momento, Daniel reagiu e foi agredido com facadas')

tokens = tokenizer.tokenize(example)
print(tokens)

['▁', 'Ú', 'l', 'tim', 'a', '▁', 'quart', 'a', '-', 'f', 'eira', '▁', '(4)', ',', '▁', 'gui', 'a', '▁', 'tur', 'í', 'sti', 'co', '▁mort', 'o', '▁no', '▁Centr', 'o', '▁do', '▁Rio', '▁', 'a', 'o', '▁', 'rea', 'gir', '▁', 'a', '▁as', 's', 'al', 'to', '▁', 'pra', 'tica', 'd', 'o', '▁por', '▁du', 'a', 's', '▁', 'mul', 'he', 're', 's', '.', '▁Image', 'n', 's', '▁das', '▁câ', 'mer', 'a', 's', '▁de', '▁se', 'gur', 'an', 'ça', '▁do', '▁local', '▁most', 'ram', '▁Daniel', '▁Mas', 'car', 'en', 'has', '▁', 'X', 'a', 'vier', '▁da', '▁Silva', ',', '▁de', '▁31', '▁an', 'o', 's', ',', '▁ser', '▁', 'abord', 'a', 'd', 'o', '▁pe', 'la', 's', '▁du', 'a', 's', '▁', 'mul', 'he', 're', 's', '▁que', '▁est', 'ava', 'm', '▁', 'e', 'm', '▁um', 'a', '▁', 'moto', 'cic', 'let', 'a', '.', '▁El', 'a', 's', '▁pe', 'gar', 'am', '▁su', 'a', '▁', 'bol', 's', 'a', '▁', 'e', '▁', 'o', 's', '▁perte', 'nce', 's', '▁que', '▁', 'a', '▁', 'v', 'í', 'tim', 'a', '▁', 't', 'in', 'ha', '▁no', '▁', 'bol', 's', 'o', '.', '▁Le', 'i', '

In [ ]:
encoded_ids = tokenizer.encode(example, add_special_tokens=False)
# print("IDs de token:", encoded_ids)
texto_decodificado = tokenizer.decode(encoded_ids)
print("Texto decodificado:", texto_decodificado)

Texto decodificado: <unk>ltima quarta-feira (4), guia tur<unk>stico morto no Centro do Rio ao reagir a assalto praticado por duas mulheres. Imagens das câmeras de segurança do local mostram Daniel Mascarenhas Xavier da Silva, de 31 anos, ser abordado pelas duas mulheres que estavam em uma motocicleta. Elas pegaram sua bolsa e os pertences que a v<unk>tima tinha no bolso. Leia mais Neste momento, Daniel reagiu e foi agredido com facadas


In [ ]:
print("Tipo de tokenizador: ", type(tokenizer))
print("Tamanho do vocabulário: ", tokenizer.vocab_size)

Tipo de tokenizador:  <class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>
Tamanho do vocabulário:  32100


In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("PamelaBorelli/XLSum_portuguese")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/PamelaBorelli___json/PamelaBorelli--XLSum_portuguese-6286fff8cf940237/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
'''
A primeira coisa que precisamos fazer é transformar o dataset em um iterador) de listas de textos
— por exemplo, uma lista de lista de textos. Usando lista de textos irá habilitar o nosso tokenizador
para funcionar mais rapidamente (treinando em lotes de textos ao invés de processar individualmente os
textos, um por vez), e deve ser um iterador se quisermos evitar ter tudo na memória de uma vez.
Se o teu corpus for grande, você vai querer aproveitar o fato de que 🤗 Datasets não carrega tudo na
memória RAM, mas armazena os elementos do dataset no disco.

'''

def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["text"]

training_corpus = get_training_corpus()

In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
example = ('saúde notícia maçã promoção pedido até às instâncias pódio')

tokens = old_tokenizer.tokenize(example)
print(tokens)

['▁', 's', 'a', 'ú', 'de', '▁not', 'í', 'cia', '▁ma', 'ç', 'ã', '▁promo', 'ç', 'ã', 'o', '▁pe', 'did', 'o', '▁at', 'é', '▁', 'à', 's', '▁in', 's', 't', 'ân', 'cia', 's', '▁', 'p', 'ó', 'd', 'i', 'o']


In [ ]:
old_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("saúde notícia maçã promoção pedido até às instâncias pódio")

[('▁saúde', (0, 5)),
 ('▁notícia', (5, 13)),
 ('▁maçã', (13, 18)),
 ('▁promoção', (18, 27)),
 ('▁pedido', (27, 34)),
 ('▁até', (34, 38)),
 ('▁às', (38, 41)),
 ('▁instâncias', (41, 52)),
 ('▁pódio', (52, 58))]

In [ ]:
'''
Vamos treinar um novo tokenizador e ver se isso resolve esses problemas.
Para isso, iremos utilizar o método train_new_from_iterator():
'''

tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 32100)

In [ ]:
tokens = tokenizer.tokenize(example)
tokens

['▁saúde',
 '▁notícia',
 '▁maçã',
 '▁promoção',
 '▁pedido',
 '▁até',
 '▁',
 'à',
 's',
 '▁instâncias',
 '▁pódio']

In [ ]:
tokenizer.save_pretrained("tokenizador_flan_treinado_XLSum_portuguese")

('tokenizador_flan_treinado_XLSum_portuguese/tokenizer_config.json',
 'tokenizador_flan_treinado_XLSum_portuguese/special_tokens_map.json',
 'tokenizador_flan_treinado_XLSum_portuguese/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer.push_to_hub("tokenizador_flan_treinado_XLSum_portuguese")

CommitInfo(commit_url='https://huggingface.co/PamelaBorelli/tokenizador_flan_treinado_XLSum_portuguese/commit/a85383c3bc2516a90f5a548592963fcfda75646d', commit_message='Upload tokenizer', commit_description='', oid='a85383c3bc2516a90f5a548592963fcfda75646d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# import torch
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#model_name = 'google/flan-t5-base'
# novo_tokenizer = AutoTokenizer.from_pretrained()
#tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
# model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")
tokenizer = AutoTokenizer.from_pretrained("PamelaBorelli/tokenizador_flan_treinado_XLSum_portuguese")
old_tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Verificando e adicionando o token de padding se necessário
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))

input_text = 'O estudo constata que 5% da população espanhola, em nível nacional, teve algum contato com o vírus, ou seja, pouco mais de 2 milhões de pessoas", declarou o ministro da Saúde, Salvador Illa, ao comentar os primeiros resultados parciais desse estudo iniciado em 27 de abril e que testou mais de 60 mil pessoas. "O resultado que mais chamou atenção foi a grande variabilidade geográfica", ressaltou o ministro. O estudo demonstra que o coronavírus circulou muito mais no centro do país, especialmente na capital, do que em outras regiões. As maiores taxas de contágio foram registradas nas vizinhas Castilla e León, com 14,2% em Soria e Castilla-La Mancha por exemplo. Sem imunidade de rebanho Até agora, o governo espanhol relatou apenas os resultados de uma primeira onda de "testes rápidos" para encontrar anticorpos contra o coronavírus. "Não encontramos diferenças entre homens e mulheres, nem nas faixas etárias", embora "seja verdade que crianças e adolescentes pareçam ter uma prevalência mais baixa", explicou o ministro.'

input = tokenizer(input_text, return_tensors="pt")

print(tokenizer.convert_ids_to_tokens(input['input_ids'][0]))
print(len(input['input_ids'][0]))

output = model.generate(input['input_ids'])

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

['▁O', '▁estudo', '▁constata', '▁que', '▁5%', '▁da', '▁população', '▁espanhola', ',', '▁em', '▁nível', '▁nacional,', '▁teve', '▁algum', '▁contato', '▁com', '▁', 'o', '▁vírus', ',', '▁ou', '▁seja', ',', '▁pouco', '▁mais', '▁de', '▁2', '▁milhões', '▁de', '▁pessoas', '",', '▁declarou', '▁', 'o', '▁ministro', '▁da', '▁Saúde', ',', '▁Salvador', '▁Il', 'la,', '▁', 'a', 'o', '▁comentar', '▁', 'os', '▁primeiros', '▁resultados', '▁parciais', '▁desse', '▁estudo', '▁iniciado', '▁em', '▁27', '▁de', '▁abril', '▁', 'e', '▁que', '▁', 'testou', '▁mais', '▁de', '▁60', '▁mil', '▁pessoas.', '▁"O', '▁resultado', '▁que', '▁mais', '▁chamou', '▁atenção', '▁foi', '▁', 'a', '▁grande', '▁varia', 'bilidade', '▁geográfica', '",', '▁ressaltou', '▁', 'o', '▁ministro.', '▁O', '▁estudo', '▁demonstra', '▁que', '▁', 'o', '▁coronavírus', '▁circulou', '▁muito', '▁mais', '▁no', '▁centro', '▁do', '▁país,', '▁especialmente', '▁na', '▁capital', ',', '▁do', '▁que', '▁em', '▁outras', '▁regiões', '.', '▁As', '▁maiores', '▁taxas

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


o,,,,


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PamelaBorelli/tokenizador_flan_treinado_XLSum_portuguese")
print("Tamanho do vocabulário: ", tokenizer.vocab_size)

Tamanho do vocabulário:  32100


In [ ]:
tokens = tokenizer.tokenize(input_text)
tokens

['▁O',
 '▁estudo',
 '▁constata',
 '▁que',
 '▁5%',
 '▁da',
 '▁população',
 '▁espanhola',
 ',',
 '▁em',
 '▁nível',
 '▁nacional,',
 '▁teve',
 '▁algum',
 '▁contato',
 '▁com',
 '▁',
 'o',
 '▁vírus',
 ',',
 '▁ou',
 '▁seja',
 ',',
 '▁pouco',
 '▁mais',
 '▁de',
 '▁2',
 '▁milhões',
 '▁de',
 '▁pessoas',
 '",',
 '▁declarou',
 '▁',
 'o',
 '▁ministro',
 '▁da',
 '▁Saúde',
 ',',
 '▁Salvador',
 '▁Il',
 'la,',
 '▁',
 'a',
 'o',
 '▁comentar',
 '▁',
 'os',
 '▁primeiros',
 '▁resultados',
 '▁parciais',
 '▁desse',
 '▁estudo',
 '▁iniciado',
 '▁em',
 '▁27',
 '▁de',
 '▁abril',
 '▁',
 'e',
 '▁que',
 '▁',
 'testou',
 '▁mais',
 '▁de',
 '▁60',
 '▁mil',
 '▁pessoas.',
 '▁"O',
 '▁resultado',
 '▁que',
 '▁mais',
 '▁chamou',
 '▁atenção',
 '▁foi',
 '▁',
 'a',
 '▁grande',
 '▁varia',
 'bilidade',
 '▁geográfica',
 '",',
 '▁ressaltou',
 '▁',
 'o',
 '▁ministro.',
 '▁O',
 '▁estudo',
 '▁demonstra',
 '▁que',
 '▁',
 'o',
 '▁coronavírus',
 '▁circulou',
 '▁muito',
 '▁mais',
 '▁no',
 '▁centro',
 '▁do',
 '▁país,',
 '▁especialmente',
 

## Treinando tokenizador unigram

In [ ]:
corpus = [
    "Este é o Curso do Abraço de Rosto.",
    "Este capítulo é sobre tokenização.",
    "Esta seção mostra vários algoritmos de tokenização.",
    "Espero que você seja capaz de entender como eles são treinados e gerar tokens."
]

In [ ]:
# model_name = 'google/flan-t5-base'
# original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("PamelaBorelli/tokenizador_flan_treinado_XLSum_portuguese")

In [ ]:
#Contando o número de ocorrências de cada palavra no corpus

from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁Este': 2,
             '▁é': 2,
             '▁o': 1,
             '▁Curso': 1,
             '▁do': 1,
             '▁Abraço': 1,
             '▁de': 3,
             '▁Rosto.': 1,
             '▁capítulo': 1,
             '▁sobre': 1,
             '▁tokenização.': 2,
             '▁Esta': 1,
             '▁seção': 1,
             '▁mostra': 1,
             '▁vários': 1,
             '▁algoritmos': 1,
             '▁Espero': 1,
             '▁que': 1,
             '▁você': 1,
             '▁seja': 1,
             '▁capaz': 1,
             '▁entender': 1,
             '▁como': 1,
             '▁eles': 1,
             '▁são': 1,
             '▁treinados': 1,
             '▁e': 1,
             '▁gerar': 1,
             '▁tokens.': 1})

In [ ]:
# Iniciando o vocabulário com uma quantidade maior do que queremos no final, incluindo caracteres
# que queremos, para as maiores substring vai permanecer as de maior ocorrência

char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

[('st', 5),
 ('os', 5),
 ('en', 5),
 ('▁E', 4),
 ('▁Es', 4),
 ('Es', 4),
 ('▁d', 4),
 ('de', 4),
 ('to', 4),
 ('▁s', 4)]

In [ ]:
# Agrupando os caracteres com as melhores subwords para chegar no vocabulario de 32100
token_freqs = list(char_freqs.items()) + sorted_subwords[: 32100 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

In [ ]:
# computando a soma de todas as frequencias, convertendo as frequencias em probabilidades.
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [ ]:
# usando o algoritmo de Viterbi que computa os melhores segmentos de cada substrings de casa palavra
# e salva na variavel best_segmentation

def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

In [ ]:
# testando algumas palavras
print(encode_word("seção", model))
print(encode_word("capítulo", model))
print(encode_word("tokenização", model))
print(encode_word("algoritmo", model))

(['seção'], 7.727431724850855)
(['capítulo'], 7.727431724850855)
(['tokenização'], 7.0342845442909105)
(['algoritmo'], 7.727431724850855)


In [ ]:
# Computando a perda do modelo no corpus
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

In [ ]:
compute_loss(model)

254.17934640689683

In [ ]:
# computando a perda para o modelo
import copy
def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

In [ ]:
scores = compute_scores(model)
print(scores["ção"])
print(scores["pí"])

0.0
0.0


In [ ]:
# adicionando special tokens usados pelo modelo e, em seguida, fazer um loop até
# que tenhamos podado tokens suficientes do vocabulário para atingir o tamanho desejado:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [ ]:
# aplicando pre-tokenização e usando a função encode_word
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokenize("Espero que você seja capaz de entender como eles são treinados e gerar tokens.", model)

['▁Espero',
 '▁que',
 '▁você',
 '▁seja',
 '▁capaz',
 '▁de',
 '▁entender',
 '▁como',
 '▁eles',
 '▁são',
 '▁treinados',
 '▁e',
 '▁gerar',
 '▁tokens.']

In [ ]:
print("Tamanho do vocabulário: ", tokenizer.vocab_size)

Tamanho do vocabulário:  54000
